## This script reads the paths from each 

In [ ]:
# import the libraries
import numpy as np
import pandas as pd
import re 
from Bio import SeqIO
from collections import defaultdict

In [ ]:
# Create a dataframe for Bandage Combined1
master_df= pd.read_csv('111_GraphResults/QueriesAndResultsMasterList.tsv', sep='\t')

In [ ]:
master_df

In [ ]:
#Drop the columns
master_df=master_df.drop(['BandageVSSPAligner','SPAlignerVSGraphAligner'],axis=1)

In [ ]:
master_df

In [ ]:
#Remove the + or - from the Path_SPAligner
#master_df['Path_SPAligner']=master_df['Path_SPAligner'].str.replace(r'[+-]', '', regex=True)
master_df['End_Bandage']=master_df['End_Bandage'].str.replace(r'[+-]', '', regex=True)

In [ ]:
#Add +2 to the values of Start_SPAligner to comapre with Start_GraphAligner
master_df['Start_SPAligner_GraphAligner'] = master_df['Start_SPAligner'].apply(lambda x: ', '.join(str(int(val) + 2) for val in x.split(',')))

In [ ]:
#Add +1 to the values of Start_SPAligner to comapre with Start_GraphAligner
master_df['Start_SPAligner_Bandage'] = master_df['Start_SPAligner'].apply(lambda x: ', '.join(str(int(val) + 1) for val in x.split(',')))

In [ ]:
master_df

In [ ]:
#Remove the braces from the end_Bandage
master_df['End_Bandage'] = master_df['End_Bandage'].str.replace('(', '').str.replace(')', '')
master_df

## This section compares the result from SPAligner and Bandage

In [ ]:
#Convert Path_GraphAligner, Start_GraphAligner and End_GraphAligner into int
master_df= master_df.fillna(000)
master_df['Start_GraphAligner']=master_df['Start_GraphAligner'].astype(int)
master_df['End_GraphAligner']=master_df['End_GraphAligner'].astype(int)
master_df['Path_GraphAligner']=master_df['Path_GraphAligner'].astype(int)
master_df

In [ ]:
master_df.dtypes

In [ ]:
master_df['Start_SPAligner']=master_df['Start_SPAligner'].astype(str)
#master_df['Start_SPAligner_GraphAligner']=master_df['Start_SPAligner_GraphAligner'].astype(str)
#master_df['Start_SPAligner_Bandage']=master_df['Start_SPAligner_Bandage'].astype(str)

master_df['End_SPAligner']=master_df['End_SPAligner'].astype(str)
master_df['Path_SPAligner']=master_df['Path_SPAligner'].astype(str)

In [ ]:
master_df

In [ ]:
def compare_values_Bandage_SPAligner(row):
    if row['Start_Bandage'] == row['Start_SPAligner'] and row['End_Bandage'] == row['End_SPAligner'] and row['Extracted_Path'] == row['Path_SPAligner']:
        return 'Full'
    elif row['Extracted_Path'] == row['Path_SPAligner']and row['Start_Bandage'] != row['Start_SPAligner'] and row['End_Bandage'] != row['End_SPAligner']:
        return 'MatchOnPath'
    elif row['Extracted_Path'] == row['Path_SPAligner'] and row['Start_Bandage'] == row['Start_SPAligner'] and row['End_Bandage'] != row['End_SPAligner'] :
        return 'MatchOnPathAndStart'
    elif row['Extracted_Path'] == row['Path_SPAligner'] and row['Start_Bandage'] != row['Start_SPAligner'] and row['End_Bandage'] == row['End_SPAligner'] :
        return 'MatchOnPathAndEnd'
    else:
        return 'Different'

In [ ]:
master_df['BandageVSSPAligner'] = master_df.apply(compare_values_Bandage_SPAligner, axis=1)

In [ ]:
master_df.dtypes

In [ ]:
master_df.to_csv("TestCompare1.tsv",sep='\t')

In [ ]:
BandagevsSPAligner_df=master_df[['Query','Path_Bandage','Start_Bandage','End_Bandage','Extracted_Path',
                                'Start_SPAligner','End_SPAligner','Path_SPAligner']]

In [ ]:
BandagevsSPAligner_df

In [ ]:
BandagevsSPAligner_df['BandageVSSPAligner'] = BandagevsSPAligner_df.apply(compare_values_Bandage_SPAligner, axis=1)

In [ ]:
BandagevsSPAligner_df.to_csv('TestRes.tsv',sep='\t')

In [ ]:
BandagevsSPAligner_df.dtypes

In [ ]:
output_list = []

for index, row in BandagevsSPAligner_df.iterrows():
    path_values = str(row['Extracted_Path']).split(',')
    start_values = str(row['Start_Bandage']).split(',')
    end_values = str(row['End_Bandage']).split(',')
    
    path_value = row['Path_SPAligner']
    start_value= str(row['Start_SPAligner']).split(',')
    end_value= str(row['End_SPAligner']).split(',')
    
    if path_value in path_values  and start_value in start_values and end_value in end_values:
        output_list.append('SinglePathFull')
    elif path_value in path_values and start_value in start_values and end_value not in end_values:
        output_list.append('SinglePathStartMatch')
    elif path_value in path_values and start_value not in start_values and end_value in end_values:
        output_list.append('SinglePathEndMatch')
    elif path_value in path_values and start_value not in start_values and end_value not in end_values:
        output_list.append('SinglePathMatch')
    elif path_value not in path_values and start_value in start_values and end_value not in end_values:
        output_list.append('SingleStartMatch')
    else:
         output_list.append('Different')

BandagevsSPAligner_df['ResultsBandageVSSPAligner'] = output_list

In [ ]:
def determine_final_result(row):
    if row['BandageVSSPAligner'] == 'Full':
        return 'Full'
    elif row['BandageVSSPAligner'] == 'MatchOnPath':
        return 'MatchOnPath'
    else:
        return row['ResultsBandageVSSPAligner']

In [ ]:
BandagevsSPAligner_df.dtypes

In [ ]:
BandagevsSPAligner_df['FinalResultBandageVSSPAligner'] = BandagevsSPAligner_df.apply(determine_final_result, axis=1)

In [ ]:
BandagevsSPAligner_df.to_csv('TestResultCategoriesBandageSPAligner.tsv',sep='\t')

## This section compares the values from GraphAligner and Bandage

In [ ]:
master_df['Start_GraphAligner']=master_df['Start_GraphAligner'].astype(str)
master_df['End_GraphAligner']=master_df['End_GraphAligner'].astype(str)
master_df['Path_GraphAligner']=master_df['Path_GraphAligner'].astype(str)

In [ ]:
BandagevsGraphAligner_df=master_df[['Query','Path_Bandage','Start_Bandage','End_Bandage','Extracted_Path',
                                'Start_GraphAligner','End_GraphAligner','Path_GraphAligner']]

In [ ]:
BandagevsGraphAligner_df


In [ ]:
def compare_values_Bandage_GraphAligner(row):
    if row['Start_Bandage'] == row['Start_GraphAligner'] and row['End_Bandage'] == row['End_GraphAligner'] and row['Extracted_Path'] == row['Path_GraphAligner']:
        return 'Full'
    elif row['Extracted_Path'] == row['Path_GraphAligner']  and row['Start_Bandage'] != row['Start_GraphAligner'] and row['End_Bandage'] != row['End_GraphAligner'] :
        return 'MatchOnPath'
    elif row['Extracted_Path'] == row['Path_GraphAligner'] and row['Start_Bandage'] == row['Start_GraphAligner'] and row['End_Bandage'] != row['End_GraphAligner'] :
        return 'MatchOnPathAndStart'
    elif row['Extracted_Path'] == row['Path_GraphAligner'] and row['Start_Bandage'] != row['Start_GraphAligner'] and row['End_Bandage'] == row['End_GraphAligner'] :
        return 'MatchOnPathAndEnd'
    else:
        return 'Different'

In [ ]:
BandagevsGraphAligner_df['BandageVSGraphAligner'] = BandagevsGraphAligner_df.apply(compare_values_Bandage_GraphAligner, axis=1)

In [ ]:
BandagevsGraphAligner_df.to_csv('rESULT44.tsv',sep='\t')

In [ ]:
output_list1 = []

for index, row in BandagevsGraphAligner_df.iterrows():
    path_values = str(row['Extracted_Path']).split(',')
    start_values = str(row['Start_Bandage']).split(',')
    end_values = str(row['End_Bandage']).split(',')
    
    path_value = str(row['Path_GraphAligner']).split(',')
    start_value= str(row['Start_GraphAligner']).split(',')
    end_value= str(row['End_GraphAligner']).split(',')
    
    if path_value in path_values and start_value in start_values and end_value in end_values:
        output_list1.append('SinglePathFull')
    elif path_value in path_values and start_value in start_values and end_value not in end_values:
        output_list1.append('SinglePathStartMatch')
    elif path_value in path_values and start_value not in start_values and end_value  in end_values:
        output_list1.append('SinglePathEndMatch')
    elif path_value in path_values and start_value not in start_values and end_value not in end_values:
        output_list1.append('SinglePathMatch')
    else:
         output_list1.append('Different')
    

BandagevsGraphAligner_df['ResultsBandageVSGraphAligner'] = output_list1

In [ ]:
BandagevsGraphAligner_df.to_csv('rESULT445.tsv',sep='\t')

In [ ]:
def determine_final_result_1(row):
    if row['BandageVSGraphAligner'] == 'Full':
        return 'Full'
    elif row['BandageVSGraphAligner'] == 'MatchOnPath':
        return 'MatchOnPath'
    else:
        return row['ResultsBandageVSGraphAligner']

In [ ]:
BandagevsGraphAligner_df['FinalResultBandageVSGraphAligner'] = BandagevsGraphAligner_df.apply(determine_final_result_1, axis=1)

In [ ]:
BandagevsGraphAligner_df.to_csv('TestResultCategoriesBandageGraphAligner.tsv',sep='\t')

In [ ]:
BandagevsGraphAligner_df

## This section compares the results from GraphAligner and SPAligner

In [ ]:
SPAlignervsGraphAligner_df=master_df[['Query','Start_SPAligner','End_SPAligner','Path_SPAligner',
                                'Start_GraphAligner','End_GraphAligner','Path_GraphAligner']]

In [ ]:
SPAlignervsGraphAligner_df.dtypes

In [ ]:
def compare_values_GraphAligner_SPAligner(row):
    if row['Start_SPAligner'] == row['Start_GraphAligner'] and row['End_SPAligner'] == row['End_GraphAligner'] and row['Path_SPAligner'] == row['Path_GraphAligner']:
        return 'Full'
    elif row['Path_SPAligner'] == row['Path_GraphAligner']and row['Start_SPAligner'] != row['Start_GraphAligner']and row['End_SPAligner'] == row['End_GraphAligner'] :
        return 'MatchOnPathAndEnd'
    elif row['Path_SPAligner'] == row['Path_GraphAligner'] and row['Start_SPAligner'] == row['Start_GraphAligner'] and row['End_SPAligner'] != row['End_GraphAligner'] :
        return 'MatchOnPathAndStart'
    elif row['Path_SPAligner'] == row['Path_GraphAligner'] and row['Start_SPAligner'] != row['Start_GraphAligner'] and row['End_SPAligner'] != row['End_GraphAligner'] :
        return 'MatchOnPath'
    else:
        return 'Different'

In [ ]:
SPAlignervsGraphAligner_df['SPAlignerVSGraphAligner'] = SPAlignervsGraphAligner_df.apply(compare_values_GraphAligner_SPAligner, axis=1)

In [ ]:
SPAlignervsGraphAligner_df.to_csv('TestResultCategoriesGraphAlignerSPAligner.tsv',sep='\t')

In [ ]:


# Assuming your three data frames are named df1, df2, and df3
# Merge df1 and df2 based on the common column "Query"
merged_df_cat = pd.merge(BandagevsSPAligner_df, BandagevsGraphAligner_df, on='Query', how='inner')

# Merge the result with df3 based on the common column "Query"
final_df = pd.merge(merged_df_cat, SPAlignervsGraphAligner_df, on='Query', how='inner')


In [ ]:
final_df.columns


In [ ]:
columns_to_drop = [ 'BandageVSSPAligner', 'ResultsBandageVSSPAligner','BandageVSGraphAligner',
       'ResultsBandageVSGraphAligner']

In [ ]:
final_df = final_df.drop(columns=columns_to_drop)

In [ ]:
final_df.to_csv('Final_df_categrories_111.tsv',sep='\t')